In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../../datasets/medical-imaging/input/metadata.csv")

In [3]:
df.head()

,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,went_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 29
0,2,0.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
3,2,6.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
4,4,0.0,F,52.0,Pneumonia/Viral/COVID-19,Y,NaN,N,N,N,...,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   patientid               950 non-null    object 
 1   offset                  697 non-null    float64
 2   sex                     870 non-null    object 
 3   age                     713 non-null    float64
 4   finding                 950 non-null    object 
 5   RT_PCR_positive         593 non-null    object 
 6   survival                361 non-null    object 
 7   intubated               248 non-null    object 
 8   intubation_present      250 non-null    object 
 9   went_icu                397 non-null    object 
 10  in_icu                  335 non-null    object 
 11  needed_supplemental_O2  90 non-null     object 
 12  extubated               37 non-null     object 
 13  temperature             78 non-null     float64
 14  pO2_saturation          119 non-null    fl

In [5]:
dfn = df.copy()

In [6]:
dfn.drop(columns=['extubated', 'in_icu', 'intubation_present', 'offset', 'doi', 'url', 'license', 'Unnamed: 29'], inplace=True)

In [7]:
mode_sex = dfn.groupby("patientid")['sex'].first().mode()[0]
dfn['sex'].fillna(mode_sex, inplace=True)

C:\Users\sharm\AppData\Local\Temp\ipykernel_11180\166216948.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfn['sex'].fillna(mode_sex, inplace=True)


In [8]:
def remove_outliers_and_impute(df, columns, threshold=1.5, impute_method='mean'):
    result_df = df.copy()
    
    for col in columns:
        # Calculate quartiles and IQR without null values
        Q1 = result_df[col].quantile(0.25)
        Q3 = result_df[col].quantile(0.75)
        IQR = Q3 - Q1

        # Calculate the lower and upper bounds for trimming
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR

        # Remove outliers from non-null values
        result_df.loc[~result_df[col].isnull(), col] = result_df.loc[~result_df[col].isnull(), col].apply(
            lambda x: x if lower_bound <= x <= upper_bound else None
        )

        # Impute null values
        if impute_method == 'mean':
            impute_value = round(result_df[col].mean(), 2)
        elif impute_method == 'median':
            impute_value = result_df[col].median()
        else:
            impute_value = impute_method
        result_df[col].fillna(impute_value, inplace=True)

    return result_df

columns = ['age', 'temperature', 'pO2_saturation', 'leukocyte_count', 'neutrophil_count', 'lymphocyte_count']
dfn = remove_outliers_and_impute(dfn, columns)

C:\Users\sharm\AppData\Local\Temp\ipykernel_11180\613074680.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result_df[col].fillna(impute_value, inplace=True)
C:\Users\sharm\AppData\Local\Temp\ipykernel_11180\613074680.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

In [9]:
dfn['age'] = dfn['age'].round()

In [10]:
dfn['clinical_notes'].fillna('Not specified', inplace=True)

C:\Users\sharm\AppData\Local\Temp\ipykernel_11180\1340761562.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfn['clinical_notes'].fillna('Not specified', inplace=True)


In [11]:
dfn['other_notes'].fillna('Not specified', inplace=True)

C:\Users\sharm\AppData\Local\Temp\ipykernel_11180\3922287833.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfn['other_notes'].fillna('Not specified', inplace=True)


In [12]:
dfn['location'].fillna('Not specified', inplace=True)

C:\Users\sharm\AppData\Local\Temp\ipykernel_11180\2572072578.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfn['location'].fillna('Not specified', inplace=True)


In [13]:
dfn['finding'] = dfn['finding'].replace('todo', 'Unknown')
dfn['RT_PCR_positive'] = dfn['RT_PCR_positive'].replace('Y', 'Yes')
dfn['RT_PCR_positive'] = dfn['RT_PCR_positive'].replace('N', 'No')
dfn['RT_PCR_positive'].fillna("Unknown", inplace=True)
dfn['survival'] = dfn['survival'].replace('Y', 'Yes')
dfn['survival'] = dfn['survival'].replace('N', 'No')
dfn['survival'].fillna("Unknown", inplace=True)
dfn['intubated'] = dfn['intubated'].replace('Y', 'Yes')
dfn['intubated'] = dfn['intubated'].replace('N', 'No')
dfn['intubated'].fillna("Unknown", inplace=True)
dfn['went_icu'] = dfn['went_icu'].replace('Y', 'Yes')
dfn['went_icu'] = dfn['went_icu'].replace('N', 'No')
dfn['went_icu'].fillna("Unknown", inplace=True)
dfn['needed_supplemental_O2'] = dfn['needed_supplemental_O2'].replace('Y', 'Yes')
dfn['needed_supplemental_O2'] = dfn['needed_supplemental_O2'].replace('N', 'No')
dfn['needed_supplemental_O2'].fillna("Unknown", inplace=True)

C:\Users\sharm\AppData\Local\Temp\ipykernel_11180\14430453.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfn['RT_PCR_positive'].fillna("Unknown", inplace=True)
C:\Users\sharm\AppData\Local\Temp\ipykernel_11180\14430453.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [14]:
def one_hot_encode_and_drop(df, column):
    # Perform one-hot encoding
    encoded_df = pd.get_dummies(df[column], prefix=column)

    # Concatenate the encoded DataFrame with the original DataFrame
    df = pd.concat([df, encoded_df], axis=1)

    # Drop the original column
    df.drop(columns=[column], inplace=True)

    return df

columns = ['finding', 'RT_PCR_positive', 'survival', 'intubated', 'went_icu', 'needed_supplemental_O2']

for col in columns:
    dfn = one_hot_encode_and_drop(dfn, col)

In [15]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 55 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   patientid                                        950 non-null    object 
 1   sex                                              950 non-null    object 
 2   age                                              950 non-null    float64
 3   temperature                                      950 non-null    float64
 4   pO2_saturation                                   950 non-null    float64
 5   leukocyte_count                                  950 non-null    float64
 6   neutrophil_count                                 950 non-null    float64
 7   lymphocyte_count                                 950 non-null    float64
 8   view                                             950 non-null    object 
 9   modality                        

In [16]:
date_formats_unique = [
    '%B %d, %Y',    # January 22, 2020
    '%Y',           # 2017
    '%b %d, %Y',    # Mar 3, 2020
    '%m/%d/%Y',     # 3/7/2020
    '%B %d',        # January 13
    '%B %Y',        # March 2003
]
def standardize_dates(date):
    for date_format in date_formats_unique:
        try:
            standardized_date = pd.to_datetime(date, format=date_format)
            if not pd.isnull(standardized_date):
                return standardized_date.strftime('%Y-%m-%d')
        except:
            pass
    # If none of the formats match, return None
    return None
dfn['date_n'] = dfn['date'].apply(standardize_dates)
dfn['year'] = pd.to_datetime(dfn['date_n']).dt.year
dfn.drop(columns=['date_n', 'date'], inplace=True)

In [17]:
dfn.to_csv('../../datasets/medical-imaging/output/transformed_metadata.csv', index=False)